In [27]:
import pandas as pd
import numpy as np
import os

In [10]:
file_job = r'../data/raw/AI_JOB_TRENDS.csv'
file_popularity = r'../data/raw/AI_ML_popularity.csv'
file_rise = r'../data/raw/The_Rise_of_AI.csv'

In [ ]:
df_job = pd.read_csv(file_job, encoding='utf-8')
df_popularity = pd.read_csv(file_popularity, encoding='latin1')
df_rise = pd.read_csv(file_rise, encoding='utf-8')

In [12]:
# 2. Clean AI_JOB_TRENDS.csv
# =========================
# Drop irrelevant columns
cols_to_drop = ['job_id', 'company_name', 'job_description_length', 'application_deadline', 'required_skills']
df_job.drop(columns=[c for c in cols_to_drop if c in df_job.columns], inplace=True)

In [13]:
# Convert salary_usd to numeric
df_job['salary_usd'] = pd.to_numeric(df_job['salary_usd'], errors='coerce')

In [14]:
# Handle missing values
df_job.fillna({'salary_usd': df_job['salary_usd'].median(), 'benefits_score': df_job['benefits_score'].median()}, inplace=True)
df_job.dropna(inplace=True)

In [15]:
# Convert posting_date to datetime if exists
if 'posting_date' in df_job.columns:
    df_job['posting_date'] = pd.to_datetime(df_job['posting_date'], errors='coerce')

In [16]:
# Encode categorical columns
cat_cols_job = ['job_title', 'experience_level', 'employment_type', 'company_location', 'company_size', 'employee_residence', 'education_required', 'industry']
df_job = pd.get_dummies(df_job, columns=cat_cols_job, drop_first=True)

In [17]:
# 3. Clean AI_ML_popularity.csv
# =========================
# Rename columns for simplicity
df_popularity.rename(columns={
    'Ai and ML(Popularity)': 'CountryPopularity',
    'Ai and ML(Popularity).1': 'CityPopularity'
}, inplace=True)

In [25]:
# Handle missing numeric values
for col in ['CountryPopularity', 'CityPopularity', 'Popularity', 'Popularity.1']:
    if col in df_popularity.columns:
        df_popularity[col] = pd.to_numeric(df_popularity[col], errors='coerce')
        df_popularity.fillna({col:df_popularity[col].median()}, inplace=True)

In [19]:
# Drop irrelevant text-heavy columns (like Rising Searches if modeling numeric)
df_popularity.drop(columns=['Top (Searches)', 'Rising (Searches)', 'Rising (Searches) Percentage',
                            'Top (Searches).1', 'Rising (Searches).1', 'Rising (Searches) Percentage.1'], errors='ignore', inplace=True)

In [20]:
# One-hot encode Country and City
df_popularity = pd.get_dummies(df_popularity, columns=['Country', 'City'], drop_first=True)

In [21]:
# 4. Clean The_Rise_of_AI.csv
# =========================
# Remove % and convert to numeric
def clean_percentage(col):
    return pd.to_numeric(col.str.replace('%','').str.strip(), errors='coerce')

for col in df_rise.columns:
    if df_rise[col].dtype == 'object':
        if df_rise[col].str.contains('%').any():
            df_rise[col] = clean_percentage(df_rise[col])

In [22]:
# Convert Year to datetime
df_rise['Year'] = pd.to_datetime(df_rise['Year'], format='%Y')

In [24]:
# Fill numeric missing with median
for col in df_rise.columns:
    if df_rise[col].dtype in [np.float64, np.int64]:
        df_rise.fillna({col: df_rise[col].median()}, inplace=True)

In [29]:
# 5. Save Cleaned Datasets
# =========================
file_path = r'../data/processed'
os.makedirs(file_path, exist_ok=True)
df_job.to_csv(os.path.join(file_path, 'cleaned_AI_JOB_TRENDS.csv'), index=False)
df_popularity.to_csv(os.path.join(file_path, 'cleaned_AI_ML_popularity.csv'), index=False)
df_rise.to_csv(os.path.join(file_path, 'cleaned_The_Rise_of_AI.csv'), index=False)